In [1]:
from bs4 import BeautifulSoup 
import pandas as pd
import os
from pymed import PubMed
import numpy as np
from selenium import webdriver
from selenium.webdriver.firefox.options import Options as FirefoxOptions
import requests

In [2]:
with open("static/react19data.html") as fp:
    # data = fp.read()
    soup = BeautifulSoup(fp, 'html.parser')

In [121]:
df2 = pd.read_csv("src/lib/react19data2.csv")
len(df2)

3580

In [4]:
papers = soup.find_all("li")
volume = []
author = []
year = []
title = []
link = []
journal = []
for paper in papers:
    author.append(paper.find_all("span", {"class": "refauthor"})[0].text)
    year.append(paper.find_all("span", {"class": "refyear"})[0].text)
    title.append(paper.find_all("span", {"class": "reftitle"})[0].text)
    link.append(paper.a['href'])
    if paper.a.find("i"):
        journal.append(paper.a.i.text)
        volume.append(paper.a.i.next_element.next_element)
    else:
        journal.append(paper.a.span.text)
        volume.append(None)


In [5]:
df = pd.DataFrame.from_dict({"author": author, "year": year, "title": title, "link": link, "journal": journal, "volume": volume})
df = df.replace("\n", "", regex=True)
get_short_title = lambda x: " ".join([f for f in x.split(" ") if len(f)][:5]) if x else "" 
df["short_title"] = df.title.apply(get_short_title)
df.head()

,author,year,title,link,journal,volume,short_title
0,"Abdelhady, M. et al.",(2023),Encephalitis following COVID-19 Vaccina...,https://www.ncbi.nlm.nih.gov/pubmed/?term=3699...,Vaccines,11,Encephalitis following COVID-19 Vaccination: A
1,"Akiyama, Y. et al.",(2023),Exacerbations of Idiopathic Systemic Ca...,https://www.ncbi.nlm.nih.gov/pubmed/?term=3708...,Intern. Med.,62:2013-2017,Exacerbations of Idiopathic Systemic Capillary
2,"Akkuzu, G. et al.",(2023),Inflammatory rheumatic diseases develop...,https://www.ncbi.nlm.nih.gov/pubmed/?term=3693...,Eur. Rev. Med. Pharmacol. Sci.,27:2143-2151,Inflammatory rheumatic diseases developed after
3,"Al-Ahmad, M. et al.",(2023),Lupus anticoagulant activity and thromb...,https://www.ncbi.nlm.nih.gov/pubmed/?term=3594...,Blood Coagul. Fibrinolysis,34:75-78,Lupus anticoagulant activity and thrombosis
4,"Alalem, N. and Yousaf, N.",(2023),A Case Report of Reactive Arthritis Aft...,https://www.ncbi.nlm.nih.gov/pubmed/?term=3700...,Cureus,15:e35544,A Case Report of Reactive


In [122]:
df2.Link = df2.Link.str.strip()
df.link = df.link.str.strip()

In [123]:
remove_whitespace = lambda x: [item for item in x if len(item)]
remove_link = lambda x: [item for item in x if not item.startswith("http")]
clean = lambda x: " ".join(remove_whitespace(remove_link(x.split(" ")))) if isinstance(x, str) and len(x) else x
clean(df2["Link Author(s)"].iloc[0])
df2["author"] = df2["Link Author(s)"].apply(clean)
df2.head()

,Link,Link Author(s),Main Category,Subcategory,Report Type,author
0,http://pubs.sciepub.com/ajmcr/9/8/3/index.html,http://pubs.sciepub.com/ajmcr/9/8/3/index.html...,Rheum-Endo-Ortho,ITP,Case Report,Parpia et al
1,https://academic.oup.com/cid/advance-article/d...,https://academic.oup.com/cid/advance-article/d...,Cardiac,Myocarditis / Pericarditis,Case Report,Perez et al
2,https://academic.oup.com/ndt/article/37/Supple...,https://academic.oup.com/ndt/article/37/Supple...,Renal,Nephropathy / IGA vasculitis,Case Report,Fornara et al
3,https://academic.oup.com/qjmed/article/114/9/6...,https://academic.oup.com/qjmed/article/114/9/6...,Cardiac,Cardiac - Other,Case Report,Lee E et al
4,https://academic.oup.com/qjmed/article/115/4/2...,https://academic.oup.com/qjmed/article/115/4/2...,Pulmonary,Pneumonia,Case Report,May et al


In [8]:

options = FirefoxOptions()
options.add_argument("--headless")
driver = webdriver.Firefox(options=options)
title2 = []
for i, link in enumerate(df2.Link):    
    try:
        driver.get(link)
        soup = BeautifulSoup(driver.page_source, "html.parser")
        print(f"{i}: {soup.title.text}")
        title = soup.title.text
    except:
        title = "NA"    
    title2.append(title)

0: Immune Thrombocytopenic Pupura after COVID-19 Vaccine, a Case Report
1: Myocarditis Following Coronavirus Disease 2019 mRNA Vaccine: A Case Series and Incidence Rate Determination | Clinical Infectious Diseases | Oxford Academic
2: MO208: De Novo Glomerulonephritides Following BNT162B2 COVID-19 Vaccine: A Case Series | Nephrology Dialysis Transplantation | Oxford Academic
3: spectrum of cardiac manifestations post Pfizer-BioNTech COVID-19 vaccination | QJM: An International Journal of Medicine | Oxford Academic
4: Eosinophilic pneumonia and COVID-19 vaccination | QJM: An International Journal of Medicine | Oxford Academic
5: Reactive Unilateral Axillary Adenopathy Following COVID-19 Vaccination – Consult QD
6: Two cases of axillary lymphadenopathy diagnosed as diffuse large B-cell lymphoma developed shortly after BNT162b2 COVID-19 vaccination. - Abstract - Europe PMC
7: 
	Immune Thrombocytopenic Purpura Associated With Pfizer-BioNT... : American Journal of Therapeutics

8: 
	Aortic 

WebDriverException: Message: Reached error page: about:neterror?e=networkProtocolError&u=https%3A//pubmed.ncbi.nlm.nih.gov/37485074/&c=UTF-8&d=Firefox%20has%20experienced%20a%20network%20protocol%20violation%20that%20cannot%20be%20repaired.
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
UnknownError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:768:5
checkReadyState@chrome://remote/content/marionette/navigate.sys.mjs:58:24
onNavigation@chrome://remote/content/marionette/navigate.sys.mjs:324:39
emit@resource://gre/modules/EventEmitter.sys.mjs:154:20
receiveMessage@chrome://remote/content/marionette/actors/MarionetteEventsParent.sys.mjs:33:25


In [124]:
df2["title"] = [""]*len(df2)
df2["title"].iloc[:len(title2)] = np.array(title2)
df2["short_title"] = df2["title"].apply(get_short_title)
df2.head()

,Link,Link Author(s),Main Category,Subcategory,Report Type,author,title,short_title
0,http://pubs.sciepub.com/ajmcr/9/8/3/index.html,http://pubs.sciepub.com/ajmcr/9/8/3/index.html...,Rheum-Endo-Ortho,ITP,Case Report,Parpia et al,Immune Thrombocytopenic Pupura after COVID-19 ...,Immune Thrombocytopenic Pupura after COVID-19
1,https://academic.oup.com/cid/advance-article/d...,https://academic.oup.com/cid/advance-article/d...,Cardiac,Myocarditis / Pericarditis,Case Report,Perez et al,Myocarditis Following Coronavirus Disease 2019...,Myocarditis Following Coronavirus Disease 2019
2,https://academic.oup.com/ndt/article/37/Supple...,https://academic.oup.com/ndt/article/37/Supple...,Renal,Nephropathy / IGA vasculitis,Case Report,Fornara et al,MO208: De Novo Glomerulonephritides Following ...,MO208: De Novo Glomerulonephritides Following
3,https://academic.oup.com/qjmed/article/114/9/6...,https://academic.oup.com/qjmed/article/114/9/6...,Cardiac,Cardiac - Other,Case Report,Lee E et al,spectrum of cardiac manifestations post Pfizer...,spectrum of cardiac manifestations post
4,https://academic.oup.com/qjmed/article/115/4/2...,https://academic.oup.com/qjmed/article/115/4/2...,Pulmonary,Pneumonia,Case Report,May et al,Eosinophilic pneumonia and COVID-19 vaccinatio...,Eosinophilic pneumonia and COVID-19 vaccination


In [126]:
import fuzzy_pandas as fpd
df2 = df2.rename({"author": "author2", "title": "title2", "short_title": "short_title2"}, axis=1)
df2 = df2.rename({}, axis=1)
merged = fpd.fuzzy_merge(df, df2,
                left_on=['short_title'],
                right_on=['short_title2'],
                ignore_case=True)
merged.head()

,author,year,title,link,journal,volume,short_title,short_author,Link,Link Author(s),Main Category,Subcategory,Report Type,author2,title2,short_title2
0,"Abdelhady, M. et al.",(2023),Encephalitis following COVID-19 Vaccina...,https://www.ncbi.nlm.nih.gov/pubmed/?term=3699...,Vaccines,11,Encephalitis following COVID-19 Vaccination: A,Abdelhady,https://pubmed.ncbi.nlm.nih.gov/36992160/,https://pubmed.ncbi.nlm.nih.gov/36992160/ Abd...,Neuro,Encephalitis / ADEM,Review,Abdelhady et al,Encephalitis following COVID-19 Vaccination: A...,Encephalitis following COVID-19 Vaccination: A
1,"Akiyama, Y. et al.",(2023),Exacerbations of Idiopathic Systemic Ca...,https://www.ncbi.nlm.nih.gov/pubmed/?term=3708...,Intern. Med.,62:2013-2017,Exacerbations of Idiopathic Systemic Capillary,Akiyama,https://pubmed.ncbi.nlm.nih.gov/37081683/,https://pubmed.ncbi.nlm.nih.gov/37081683/ Aki...,Immune System,Other,Case Report,Akiyama Y et al,Exacerbations of Idiopathic Systemic Capillary...,Exacerbations of Idiopathic Systemic Capillary
2,"Akkuzu, G. et al.",(2023),Inflammatory rheumatic diseases develop...,https://www.ncbi.nlm.nih.gov/pubmed/?term=3693...,Eur. Rev. Med. Pharmacol. Sci.,27:2143-2151,Inflammatory rheumatic diseases developed after,Akkuzu,https://pubmed.ncbi.nlm.nih.gov/36930514/,https://pubmed.ncbi.nlm.nih.gov/36930514/ Akk...,Rheum-Endo-Ortho,Autoimmune,Case Review,Akkuzu et al,Inflammatory rheumatic diseases developed afte...,Inflammatory rheumatic diseases developed after
3,"Al-Ahmad, M. et al.",(2023),Lupus anticoagulant activity and thromb...,https://www.ncbi.nlm.nih.gov/pubmed/?term=3594...,Blood Coagul. Fibrinolysis,34:75-78,Lupus anticoagulant activity and thrombosis,Al-Ahmad,https://pubmed.ncbi.nlm.nih.gov/35946452/,https://pubmed.ncbi.nlm.nih.gov/35946452/ Al-...,Rheum-Endo-Ortho,Lupus,Case Report,Al-Ahmad et al (2),Lupus anticoagulant activity and thrombosis po...,Lupus anticoagulant activity and thrombosis
4,"Alalem, N. and Yousaf, N.",(2023),A Case Report of Reactive Arthritis Aft...,https://www.ncbi.nlm.nih.gov/pubmed/?term=3700...,Cureus,15:e35544,A Case Report of Reactive,Alalem,https://pubmed.ncbi.nlm.nih.gov/37007315/,https://pubmed.ncbi.nlm.nih.gov/37007315/ Ala...,Rheum-Endo-Ortho,Inflammation / Arthritis,Case Report,Alalem & Yousaf,A Case Report of Reactive Arthritis After COVI...,A Case Report of Reactive


In [127]:
df["short_author"] = df["author"].apply(lambda x: x.split(",")[0])
def get_short_author(x):
    if not isinstance(x, str):
        return x
    output = x.replace(" et al", "").strip()
    res = output.split(" ")
    if len(res) > 1 and len(res[0]) > len(res[1]):
        return res[0]
    else:
        return output

df2["short_author2"] = df2["author2"].apply(get_short_author)
df2["short_author2"]

0          Parpia
1           Perez
2         Fornara
3             Lee
4             May
          ...    
3575      Mimouni
3576    Geeraerts
3577       Yousaf
3578          NaN
3579     Langerak
Name: short_author2, Length: 3580, dtype: object

In [128]:
(~df2.short_title2.isnull()).sum()

3580

In [129]:
df = df[~df.duplicated(subset=["short_title", "volume"])]
df.shape

(3263, 8)

In [130]:
not_matched = df2[~df2["Link"].isin(merged["Link"])]
print(len(not_matched))
not_matched.head()

388


,Link,Link Author(s),Main Category,Subcategory,Report Type,author2,title2,short_title2,short_author2
3,https://academic.oup.com/qjmed/article/114/9/6...,https://academic.oup.com/qjmed/article/114/9/6...,Cardiac,Cardiac - Other,Case Report,Lee E et al,spectrum of cardiac manifestations post Pfizer...,spectrum of cardiac manifestations post,Lee
5,https://consultqd.clevelandclinic.org/reactive...,https://consultqd.clevelandclinic.org/reactive...,Oncology,Lymphadenopathy / Adenopathy,Case Report,tweets&utm_medium=social&utm_source=twitter&ut...,Reactive Unilateral Axillary Adenopathy Follow...,Reactive Unilateral Axillary Adenopathy Following,tweets&utm_medium=social&utm_source=twitter&ut...
7,https://journals.lww.com/americantherapeutics/...,https://journals.lww.com/americantherapeutics/...,Rheum-Endo-Ortho,ITP,Case Report,Ghous et al,\n\tImmune Thrombocytopenic Purpura Associated...,\n\tImmune Thrombocytopenic Purpura Associated...,Ghous
8,https://journals.lww.com/americantherapeutics/...,https://journals.lww.com/americantherapeutics/...,Rheum-Endo-Ortho,Thrombolytic / Thrombocytopenia,Case Report,Pujwani et al,\n\tAortic Mural Thrombosis Diagnosed After a ...,\n\tAortic Mural Thrombosis Diagnosed After,Pujwani
9,https://journals.lww.com/jcardiovascularmedici...,https://journals.lww.com/jcardiovascularmedici...,Cardiac,Myocarditis / Pericarditis,Case Report,Ahmed T et al,\n\tA report on short-term follow-up cardiac i...,\n\tA report on short-term follow-up,Ahmed


In [131]:
df2.columns

Index(['Link', 'Link Author(s)', 'Main Category', 'Subcategory', 'Report Type',
       'author2', 'title2', 'short_title2', 'short_author2'],
      dtype='object')

In [134]:
merged_author = df.merge(not_matched, how="inner", left_on="short_author", right_on="short_author2")
# merged_author = merged_author.drop(columns=["title_y", "short_title_y"]).rename(
#     {"author_x": "author", "title_x": "title", "short_title_x": "short_title"}, axis=1)
merged_author.drop_duplicates(subset=["link"], inplace=True)
print(merged_author.shape)
merged_author.columns

(348, 17)


Index(['author', 'year', 'title', 'link', 'journal', 'volume', 'short_title',
       'short_author', 'Link', 'Link Author(s)', 'Main Category',
       'Subcategory', 'Report Type', 'author2', 'title2', 'short_title2',
       'short_author2'],
      dtype='object')

In [137]:
merged = merged.drop_duplicates(subset=["short_title", "volume"])
merged = merged.reset_index(drop=True)
merged["short_author2"] = merged["short_author"]
merged.shape

(3173, 17)

In [138]:
set(merged_author.columns.tolist()).difference(merged.columns.tolist())

set()

In [139]:
merged_author = merged_author.drop_duplicates(subset=["short_title", "volume"])
merged_author = merged_author.reset_index(drop=True)


In [140]:
merged_author.head()

,author,year,title,link,journal,volume,short_title,short_author,Link,Link Author(s),Main Category,Subcategory,Report Type,author2,title2,short_title2,short_author2
0,"Albakri, K. et al.",(2023),Characteristics of sudden hearing loss ...,https://www.ncbi.nlm.nih.gov/pubmed/?term=3759...,Eur. Arch. Otorhinolaryngol.,280:5167-5176,Characteristics of sudden hearing loss,Albakri,https://pubmed.ncbi.nlm.nih.gov/37594544/,https://pubmed.ncbi.nlm.nih.gov/37594544/ Alb...,"Ear, Nose and Throat",Hearing Loss,Review,Albakri K et al,,,Albakri
1,"Almalki, M.E. et al.",(2023),Regional Myopericarditis Mimicking Infe...,https://www.ncbi.nlm.nih.gov/pubmed/?term=3752...,Cureus,15:e41168,Regional Myopericarditis Mimicking Inferior My...,Almalki,https://www.cureus.com/articles/166935-regiona...,https://www.cureus.com/articles/166935-regiona...,Cardiac,Myocarditis / Pericarditis,Case Report,Almalki et al,,,Almalki
2,"Alsubaie, A. et al.",(2023),A 20-Year-Old Man with IgA Vasculitis f...,https://www.ncbi.nlm.nih.gov/pubmed/?term=3749...,Case Rep. Rheumatol.,2023:9505383,A 20-Year-Old Man with IgA,Alsubaie,https://pubmed.ncbi.nlm.nih.gov/37492803/,https://pubmed.ncbi.nlm.nih.gov/37492803/ Als...,Rheum-Endo-Ortho,Nephropathy / IGA Vasculitis,Case Report,Alsubaie et al,,,Alsubaie
3,"Alsudais, A.S. et al.",(2023),Autoimmune diabetes mellitus after COVI...,https://www.ncbi.nlm.nih.gov/pubmed/?term=3754...,BMC Endocr. Disord.,23:164,Autoimmune diabetes mellitus after COVID-19,Alsudais,https://pubmed.ncbi.nlm.nih.gov/37542216/,https://pubmed.ncbi.nlm.nih.gov/37542216/ Als...,Rheum-Endo-Ortho,Endocrine,Case Report,Alsudais AS et al,,,Alsudais
4,"Ang, S. et al.",(2023),Ph-Positive B-Cell Acute Lymphoblastic ...,https://www.ncbi.nlm.nih.gov/pubmed/?term=3698...,Medicina,59,Ph-Positive B-Cell Acute Lymphoblastic Leukemia,Ang,https://pubmed.ncbi.nlm.nih.gov/37198501/,https://pubmed.ncbi.nlm.nih.gov/37198501/ Ang...,Neuro,NaN,NaN,Ang T et al,Orbital inflammation following COVID-19 vaccin...,Orbital inflammation following COVID-19 vaccin...,Ang


In [141]:
merged = merged.loc[~merged.index.duplicated(keep='first')]
merged_author = merged_author.loc[~merged_author.index.duplicated(keep='first')]
merged.shape

(3173, 17)

In [142]:
merged.columns

Index(['author', 'year', 'title', 'link', 'journal', 'volume', 'short_title',
       'short_author', 'Link', 'Link Author(s)', 'Main Category',
       'Subcategory', 'Report Type', 'author2', 'title2', 'short_title2',
       'short_author2'],
      dtype='object')

In [143]:
merged_author = merged_author[~merged_author.link.isin(merged.link)]
merged_author.shape

(82, 17)

In [144]:
merged_author.columns

Index(['author', 'year', 'title', 'link', 'journal', 'volume', 'short_title',
       'short_author', 'Link', 'Link Author(s)', 'Main Category',
       'Subcategory', 'Report Type', 'author2', 'title2', 'short_title2',
       'short_author2'],
      dtype='object')

In [145]:
both_merge = pd.concat([merged, merged_author], ignore_index=True)
both_merge.head()

,author,year,title,link,journal,volume,short_title,short_author,Link,Link Author(s),Main Category,Subcategory,Report Type,author2,title2,short_title2,short_author2
0,"Abdelhady, M. et al.",(2023),Encephalitis following COVID-19 Vaccina...,https://www.ncbi.nlm.nih.gov/pubmed/?term=3699...,Vaccines,11,Encephalitis following COVID-19 Vaccination: A,Abdelhady,https://pubmed.ncbi.nlm.nih.gov/36992160/,https://pubmed.ncbi.nlm.nih.gov/36992160/ Abd...,Neuro,Encephalitis / ADEM,Review,Abdelhady et al,Encephalitis following COVID-19 Vaccination: A...,Encephalitis following COVID-19 Vaccination: A,Abdelhady
1,"Akiyama, Y. et al.",(2023),Exacerbations of Idiopathic Systemic Ca...,https://www.ncbi.nlm.nih.gov/pubmed/?term=3708...,Intern. Med.,62:2013-2017,Exacerbations of Idiopathic Systemic Capillary,Akiyama,https://pubmed.ncbi.nlm.nih.gov/37081683/,https://pubmed.ncbi.nlm.nih.gov/37081683/ Aki...,Immune System,Other,Case Report,Akiyama Y et al,Exacerbations of Idiopathic Systemic Capillary...,Exacerbations of Idiopathic Systemic Capillary,Akiyama
2,"Akkuzu, G. et al.",(2023),Inflammatory rheumatic diseases develop...,https://www.ncbi.nlm.nih.gov/pubmed/?term=3693...,Eur. Rev. Med. Pharmacol. Sci.,27:2143-2151,Inflammatory rheumatic diseases developed after,Akkuzu,https://pubmed.ncbi.nlm.nih.gov/36930514/,https://pubmed.ncbi.nlm.nih.gov/36930514/ Akk...,Rheum-Endo-Ortho,Autoimmune,Case Review,Akkuzu et al,Inflammatory rheumatic diseases developed afte...,Inflammatory rheumatic diseases developed after,Akkuzu
3,"Al-Ahmad, M. et al.",(2023),Lupus anticoagulant activity and thromb...,https://www.ncbi.nlm.nih.gov/pubmed/?term=3594...,Blood Coagul. Fibrinolysis,34:75-78,Lupus anticoagulant activity and thrombosis,Al-Ahmad,https://pubmed.ncbi.nlm.nih.gov/35946452/,https://pubmed.ncbi.nlm.nih.gov/35946452/ Al-...,Rheum-Endo-Ortho,Lupus,Case Report,Al-Ahmad et al (2),Lupus anticoagulant activity and thrombosis po...,Lupus anticoagulant activity and thrombosis,Al-Ahmad
4,"Alalem, N. and Yousaf, N.",(2023),A Case Report of Reactive Arthritis Aft...,https://www.ncbi.nlm.nih.gov/pubmed/?term=3700...,Cureus,15:e35544,A Case Report of Reactive,Alalem,https://pubmed.ncbi.nlm.nih.gov/37007315/,https://pubmed.ncbi.nlm.nih.gov/37007315/ Ala...,Rheum-Endo-Ortho,Inflammation / Arthritis,Case Report,Alalem & Yousaf,A Case Report of Reactive Arthritis After COVI...,A Case Report of Reactive,Alalem


In [147]:
both_merge = both_merge.drop_duplicates(subset=["link"])
print(both_merge.shape)
both_merge.head()

(3255, 17)


,author,year,title,link,journal,volume,short_title,short_author,Link,Link Author(s),Main Category,Subcategory,Report Type,author2,title2,short_title2,short_author2
0,"Abdelhady, M. et al.",(2023),Encephalitis following COVID-19 Vaccina...,https://www.ncbi.nlm.nih.gov/pubmed/?term=3699...,Vaccines,11,Encephalitis following COVID-19 Vaccination: A,Abdelhady,https://pubmed.ncbi.nlm.nih.gov/36992160/,https://pubmed.ncbi.nlm.nih.gov/36992160/ Abd...,Neuro,Encephalitis / ADEM,Review,Abdelhady et al,Encephalitis following COVID-19 Vaccination: A...,Encephalitis following COVID-19 Vaccination: A,Abdelhady
1,"Akiyama, Y. et al.",(2023),Exacerbations of Idiopathic Systemic Ca...,https://www.ncbi.nlm.nih.gov/pubmed/?term=3708...,Intern. Med.,62:2013-2017,Exacerbations of Idiopathic Systemic Capillary,Akiyama,https://pubmed.ncbi.nlm.nih.gov/37081683/,https://pubmed.ncbi.nlm.nih.gov/37081683/ Aki...,Immune System,Other,Case Report,Akiyama Y et al,Exacerbations of Idiopathic Systemic Capillary...,Exacerbations of Idiopathic Systemic Capillary,Akiyama
2,"Akkuzu, G. et al.",(2023),Inflammatory rheumatic diseases develop...,https://www.ncbi.nlm.nih.gov/pubmed/?term=3693...,Eur. Rev. Med. Pharmacol. Sci.,27:2143-2151,Inflammatory rheumatic diseases developed after,Akkuzu,https://pubmed.ncbi.nlm.nih.gov/36930514/,https://pubmed.ncbi.nlm.nih.gov/36930514/ Akk...,Rheum-Endo-Ortho,Autoimmune,Case Review,Akkuzu et al,Inflammatory rheumatic diseases developed afte...,Inflammatory rheumatic diseases developed after,Akkuzu
3,"Al-Ahmad, M. et al.",(2023),Lupus anticoagulant activity and thromb...,https://www.ncbi.nlm.nih.gov/pubmed/?term=3594...,Blood Coagul. Fibrinolysis,34:75-78,Lupus anticoagulant activity and thrombosis,Al-Ahmad,https://pubmed.ncbi.nlm.nih.gov/35946452/,https://pubmed.ncbi.nlm.nih.gov/35946452/ Al-...,Rheum-Endo-Ortho,Lupus,Case Report,Al-Ahmad et al (2),Lupus anticoagulant activity and thrombosis po...,Lupus anticoagulant activity and thrombosis,Al-Ahmad
4,"Alalem, N. and Yousaf, N.",(2023),A Case Report of Reactive Arthritis Aft...,https://www.ncbi.nlm.nih.gov/pubmed/?term=3700...,Cureus,15:e35544,A Case Report of Reactive,Alalem,https://pubmed.ncbi.nlm.nih.gov/37007315/,https://pubmed.ncbi.nlm.nih.gov/37007315/ Ala...,Rheum-Endo-Ortho,Inflammation / Arthritis,Case Report,Alalem & Yousaf,A Case Report of Reactive Arthritis After COVI...,A Case Report of Reactive,Alalem


In [130]:
# Pubmed stuff
# https://github.com/gijswobben/pymed/blob/master/examples/advanced_search/main.py
# last, first = df.author.iloc[0].split(",")[:2]
# first = first.replace("et al.","").strip()
# short_title = " ".join([f for f in df.title.iloc[0].split(" ") if len(f)][:3])
pubmed = PubMed(tool="MyTool", email="my@email.address")
# results = pubmed.query(f"{last}, {first}[Author] AND {short_title}", max_results=10)
# pubmed_papers = list(results)

NameError: name 'last' is not defined

In [160]:
# Google scholar stuff stuff
# https://medium.com/@nandinisaini021/scraping-publications-of-aerial-image-research-papers-on-google-scholar-using-python-a0dee9744728
# this function for the extracting information of the tags
url = "https://scholar.google.com/scholar?start={}&q=object+detection+in+aerial+image+&hl=en&as_sdt=0,5"
def get_tags(doc):
  paper_tag = doc.select('[data-lid]')
  cite_tag = doc.select('[title=Cite] + a')
  link_tag = doc.find_all('h3',{"class" : "gs_rt"})
  author_tag = doc.find_all("div", {"class": "gs_a"})

  return paper_tag,cite_# it will return the title of the paper
def get_papertitle(paper_tag):
  
  paper_names = []
  
  for tag in paper_tag:
    paper_names.append(tag.select('h3')[0].get_text())

  return paper_namestag,link_tag,author_tag
# it will return the number of citation of the paper
def get_citecount(cite_tag):
  cite_count = []
  for i in cite_tag:
    cite = i.text
    if i is None or cite is None:  # if paper has no citatation then consider 0
      cite_count.append(0)
    else:
      tmp = re.search(r'\d+', cite) # its handle the None type object error and re use to remove the string " cited by " and return only integer value
      if tmp is None :
        cite_count.append(0)
      else :
        cite_count.append(int(tmp.group()))

  return cite_count
# function for the getting link information
def get_link(link_tag):

  links = []

  for i in range(len(link_tag)) :
    links.append(link_tag[i].a['href']) 

  return links
# function for the getting autho , year and publication information
def get_author_year_publi_info(authors_tag):
  years = []
  publication = []
  authors = []
  for i in range(len(authors_tag)):
      authortag_text = (authors_tag[i].text).split()
      year = int(re.search(r'\d+', authors_tag[i].text).group())
      years.append(year)
      publication.append(authortag_text[-1])
      author = authortag_text[0] + ' ' + re.sub(',','', authortag_text[1])
      authors.append(author)
  
  return years , publication, authors
# creating final repository
paper_repos_dict = {
                    'Paper Title' : [],
                    'Year' : [],
                    'Author' : [],
                    'Citation' : [],
                    'Publication' : [],
                    'Url of paper' : [] }

# adding information in repository
def add_in_paper_repo(papername,year,author,cite,publi,link):
  paper_repos_dict['Paper Title'].extend(papername)
  paper_repos_dict['Year'].extend(year)
  paper_repos_dict['Author'].extend(author)
  paper_repos_dict['Citation'].extend(cite)
  paper_repos_dict['Publication'].extend(publi)
  paper_repos_dict['Url of paper'].extend(link)

  return pd.DataFrame(paper_repos_dict)

In [148]:
both_merge.columns

Index(['author', 'year', 'title', 'link', 'journal', 'volume', 'short_title',
       'short_author', 'Link', 'Link Author(s)', 'Main Category',
       'Subcategory', 'Report Type', 'author2', 'title2', 'short_title2',
       'short_author2'],
      dtype='object')

0       Encephalitis following COVID-19 Vaccination: A...
1       Exacerbations of Idiopathic Systemic Capillary...
2       Inflammatory rheumatic diseases developed afte...
3       Lupus anticoagulant activity and thrombosis po...
4       A Case Report of Reactive Arthritis After COVI...
                              ...                        
3250    Familial thrombocytopenia flare-up following t...
3251    Covid-19 Vaccine Injuries - Preventing Inequit...
3252            Adenopathy Following COVID-19 Vaccination
3253                 Post-vaccinal minimal change disease
3254                Herpes Zoster after COVID-vaccination
Name: title, Length: 3255, dtype: object

In [162]:
output = both_merge.drop(["Link", "short_author", "Link Author(s)", "author2", "title2", "short_title2", "short_author2"], 
                axis=1).rename({"Main Category": "main_category", "Subcategory": "sub_category", "Report Type": "report_type",
                               }, axis=1)
output.title = both_merge.title.str.replace("\s{2,}", " ", regex=True)
output.author = both_merge.author.str.replace("\s{2,}", " ", regex=True)
output.journal = both_merge.journal.str.replace("\s{2,}", " ", regex=True)
output.volume = both_merge.volume.str.replace("^\s{1,}", "", regex=True)
output.year = both_merge.year.str.replace("(", "")
output.year = output.year.str.replace(")", "")

output.to_json("src/lib/react19data_merge.json", orient="records")


In [84]:
# Export
df.to_json("static/react19data.json", orient="records")